### 📥 Carregamento da Base Processada

Este trecho localiza automaticamente a raiz do projeto e carrega o arquivo df_trein.parquet da pasta data/processed.

Após validar que o arquivo existe, o dataset é lido com pandas e são exibidos o shape e os tipos das colunas, garantindo que a base está pronta para a etapa de modelagem.

In [2]:
import pandas as pd
from pathlib import Path

# Descobre a raiz do projeto (assume que o notebook está em notbooks/)
try:
    ROOT = Path(__file__).resolve().parents[1]
except NameError:
    # __file__ não existe em notebooks; usa o cwd como base
    ROOT = Path.cwd().resolve().parent

data_path = ROOT / 'data' / 'processed' / 'df_trein.parquet'

if not data_path.exists():
    raise FileNotFoundError(f"Arquivo não encontrado: {data_path}")

df = pd.read_parquet(data_path)

print("Shape do dataset:", df.shape)
print("\nTipos de dados:\n")
print(df.dtypes)

Shape do dataset: (1390, 19)

Tipos de dados:

ANO                   int64
IDADE                 int64
FASE                  int64
DEFASAGEM             int64
IAA                 float64
IEG                 float64
IDA                 float64
IAN                 float64
IPS                 float64
IPV                 float64
NOTA_MAT            float64
NOTA_POR            float64
ABANDONO              int64
IDA_MISSING           int64
IEG_MISSING           int64
IPV_MISSING           int64
IPS_MISSING           int64
NOTA_MAT_MISSING      int64
NOTA_POR_MISSING      int64
dtype: object


### 📌 Baseline com Regressão Logística (2022 → 2023)

Este bloco treina um modelo baseline de classificação usando Regressão Logística dentro de um Pipeline, avaliando de forma temporal: treino em 2022 e teste (holdout) em 2023.

O fluxo é:

 - Sanity checks: garante que o target ABANDONO existe e que os anos no dataset são apenas 2022 e 2023.

 - Separação de features e target e divisão temporal (2022/2023).

Pipeline com:

 - RobustScaler: reduz o impacto de outliers nas variáveis numéricas.

 - LogisticRegression com class_weight="balanced": trata o desbalanceamento dando mais peso à classe de abandono.

 - Avaliação por probabilidade com métricas independentes de limiar:

 - ROC-AUC e PR-AUC no holdout 2023.

 - Avaliação com threshold 0.5 (padrão) e, adicionalmente, um threshold “ótimo” por F1 (apenas diagnóstico) para entender o trade-off entre precisão e recall, exibindo matriz de confusão e relatório de classificação em ambos os casos.

In [3]:
import pyarrow.parquet as pq

from sklearn.pipeline import Pipeline
#from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score, average_precision_score,
    confusion_matrix, classification_report,
    precision_recall_curve
)

TARGET = "ABANDONO"

# ---- sanity checks ----
assert TARGET in df.columns, f"Target {TARGET} não está no dataset"
assert set(df["ANO"].unique()) == {2022, 2023}, "Esperado ANO em {2022, 2023}"

features = [c for c in df.columns if c != TARGET]

train = df[df["ANO"] == 2022].copy()
test  = df[df["ANO"] == 2023].copy()

X_train, y_train = train[features], train[TARGET]
X_test,  y_test  = test[features],  test[TARGET]

# ---- baseline model ----
"""pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("lr", LogisticRegression(
        max_iter=2000,
        class_weight="balanced",   # importante para classe desbalanceada
        solver="lbfgs"
    ))
])"""

pipe = Pipeline([
    ("scaler", RobustScaler()),
    ("lr", LogisticRegression(
        max_iter=5000,
        class_weight="balanced",
        solver="liblinear",   # bom para L1/L2 em datasets menores
        penalty="l2"
    ))
])

pipe.fit(X_train, y_train)

proba = pipe.predict_proba(X_test)[:, 1]

# Métricas “agnósticas a limiar”
roc = roc_auc_score(y_test, proba)
ap  = average_precision_score(y_test, proba)

print(f"ROC-AUC (2023 holdout): {roc:.4f}")
print(f"PR-AUC  (2023 holdout): {ap:.4f}")

# ---- avaliação com limiar padrão 0.5 ----
pred_05 = (proba >= 0.5).astype(int)
print("\nConfusion (thr=0.50):")
print(confusion_matrix(y_test, pred_05))
print("\nReport (thr=0.50):")
print(classification_report(y_test, pred_05, digits=3))

# ---- achar limiar que maximiza F1 no holdout (apenas para diagnóstico) ----
prec, rec, thr = precision_recall_curve(y_test, proba)
f1 = 2 * prec * rec / (prec + rec + 1e-12)

best_idx = f1.argmax()
best_thr = thr[best_idx - 1] if best_idx > 0 else 0.5

print(f"\nMelhor threshold por F1 no holdout: {best_thr:.3f}")
print(f"F1={f1[best_idx]:.3f} | Precision={prec[best_idx]:.3f} | Recall={rec[best_idx]:.3f}")

pred_best = (proba >= best_thr).astype(int)
print("\nConfusion (best_thr):")
print(confusion_matrix(y_test, pred_best))
print("\nReport (best_thr):")
print(classification_report(y_test, pred_best, digits=3))

ROC-AUC (2023 holdout): 0.6532
PR-AUC  (2023 holdout): 0.4191

Confusion (thr=0.50):
[[500  32]
 [149  39]]

Report (thr=0.50):
              precision    recall  f1-score   support

           0      0.770     0.940     0.847       532
           1      0.549     0.207     0.301       188

    accuracy                          0.749       720
   macro avg      0.660     0.574     0.574       720
weighted avg      0.713     0.749     0.704       720


Melhor threshold por F1 no holdout: 0.302
F1=0.475 | Precision=0.384 | Recall=0.622

Confusion (best_thr):
[[343 189]
 [ 71 117]]

Report (best_thr):
              precision    recall  f1-score   support

           0      0.829     0.645     0.725       532
           1      0.382     0.622     0.474       188

    accuracy                          0.639       720
   macro avg      0.605     0.634     0.599       720
weighted avg      0.712     0.639     0.659       720



/home/vladm/Dev/predicao_risco_defasagem_datathon/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(


### 📊 Importância das Variáveis (Regressão Logística)

Este trecho extrai os coeficientes do modelo de Regressão Logística treinado e organiza em um DataFrame para análise.

Os coeficientes representam o impacto de cada variável no risco de abandono:

 - Coeficiente positivo → aumenta a probabilidade de abandono.

 - Coeficiente negativo → reduz a probabilidade de abandono.

As variáveis são ordenadas pelo valor absoluto do coeficiente (abs_coef), destacando as features com maior influência no modelo.

In [4]:
import numpy as np

# recupera o modelo treinado
lr = pipe.named_steps["lr"]

coef_df = pd.DataFrame({
    "feature": X_train.columns,
    "coef": lr.coef_[0]
})

# ordena por impacto absoluto
coef_df["abs_coef"] = coef_df["coef"].abs()
coef_df = coef_df.sort_values("abs_coef", ascending=False)

coef_df.head(15)

,feature,coef,abs_coef
5,IEG,-0.929160,0.929160
9,IPV,-0.403987,0.403987
17,NOTA_POR_MISSING,0.370303,0.370303
16,NOTA_MAT_MISSING,0.370303,0.370303
2,FASE,0.297828,0.297828
1,IDADE,-0.217200,0.217200
7,IAN,0.210977,0.210977
8,IPS,0.204813,0.204813
3,DEFASAGEM,-0.167195,0.167195
6,IDA,-0.131117,0.131117
